In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer


from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
cd /content/drive/MyDrive/data/pjt2

/content/drive/MyDrive/data/pjt2


In [ ]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('safe/train데이터_전처리.csv', encoding='utf-8')
train_df.head()

,Unnamed: 0,index,title,topic_idx,data,new_title,new_title1,new_title2,new_title3,new_title4,data2
0,0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,인천→핀란드 항공기 결항…휴가철 여행객 분통,인천 핀란드 항공기 결항 휴가 철 여행객 분통,인천 핀란드 항공기 결항 휴가 철 여행객 분통,인천 핀란드 항공기 결항 휴가 철 여행객 분통,인천 핀란드 항공기 결항 휴가 여행객 분통,인천 핀란드 항공기 결항 휴가 여행객 분통,인천핀란드 항공기 결항휴가철 여행객 분통
1,1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,실리콘밸리 넘어서겠다…구글 15조원 들여 미국 전역 거점화,실리콘밸리 넘어서다 구글 조원 들이다 미국 전역 거점 화,실리콘밸리 넘어서다 구글 조원 들이다 미국 전역 거점 화,실리콘밸리 넘어서다 구글 조원 들이다 미국 전역 거점 화,실리콘밸리 넘어서다 구글 조원 들이다 미국 전역 거점,실리콘밸리 넘어서다 구글 조원 들이다 미국 전역 거점,실리콘밸리 넘어서겠다구글 15조원 들여 미국 전역 거점화
2,2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,이란 외무 긴장 완화 해결 책 은 미국 경제 전쟁 멈추다,이란 외무 긴장 완화 해결 책 은 미국 경제 전쟁 멈추다,이란 외무 긴장 완화 해결 책 은 미국 경제 전쟁 멈추다,이란 외무 긴장 완화 해결 미국 경제 전쟁 멈추다,이란 외무 긴장 완화 해결 미국 경제 전쟁 멈추다,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것
3,3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,NYT 클린턴 측근 한국 기업 특수관계 조명…공과 사 맞물려종합,NYT 클린턴 측근 한국 기업 특수 관계 조명 공과 맞다 물리다 종합,NYT 클린턴 측근 한국 기업 특수 관계 조명 공과 사 맞다 물리다 종합,클린턴 측근 한국 기업 특수 관계 조명 공과 맞다 물리다 종합,클린턴 측근 한국 기업 특수 관계 조명 공과 맞다 물리다 종합,NYT 클린턴 측근 한국 기업 특수 관계 조명 공과 맞다 물리다 종합,NYT 클린턴 측근 한국 기업 특수관계 조명공과 사 맞물려종합
4,4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,시진핑 트럼프 중미 무역 협상 조속 타결 희망,시진핑 트럼프 에 중미 무역 협상 조속 타결 희망,시진핑 트럼프 중미 무역 협상 조속 타결 희망,시진핑 트럼프 중미 무역 협상 조속 타결 희망,시진핑 트럼프 중미 무역 협상 조속 타결 희망,시진핑 트럼프에 중미 무역협상 조속 타결 희망


In [ ]:
#for i in range(len(train_df)):
#  train_df['new_title4'][i]=train_df['new_title4'][i][1:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
train_dataset = []
for sen, label in zip(train_df['data2'], train_df['topic_idx']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(int(label)))

  train_dataset.append(data_train)
# kobert 트레인 할때 input 방식

In [ ]:
train_dataset[:5]

[['인천핀란드 항공기 결항휴가철 여행객 분통', '4'],
 ['실리콘밸리 넘어서겠다구글 15조원 들여  미국 전역 거점화', '4'],
 ['이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것', '4'],
 ['NYT 클린턴 측근 한국 기업 특수관계 조명공과 사 맞물려종합', '4'],
 ['시진핑 트럼프에 중미 무역협상 조속 타결 희망', '4']]

In [ ]:
class BERTDataset(Dataset): # 
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5
# 파라미터 설정

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model() ##kober 모델 정보 가지고오기(단어)

using cached model
using cached model


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_dataset, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.4).to(device)


#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    # test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1427 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.00006365776062 train acc 0.0
epoch 1 batch id 201 loss 1.208027720451355 train acc 0.3748445273631841
epoch 1 batch id 401 loss 0.6557317972183228 train acc 0.5938279301745636
epoch 1 batch id 601 loss 0.5074830651283264 train acc 0.6722649750415973
epoch 1 batch id 801 loss 0.3273276090621948 train acc 0.7140683520599251
epoch 1 batch id 1001 loss 0.02826971374452114 train acc 0.7562125374625375
epoch 1 batch id 1201 loss 0.44169506430625916 train acc 0.7837999583680266
epoch 1 batch id 1401 loss 0.21932154893875122 train acc 0.7933172733761599
epoch 1 train acc 0.7959781327642225


  0%|          | 0/1427 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.5758172273635864 train acc 0.8125
epoch 2 batch id 201 loss 0.44066715240478516 train acc 0.8748445273631841
epoch 2 batch id 401 loss 0.4951191246509552 train acc 0.8676745635910225
epoch 2 batch id 601 loss 0.5064229965209961 train acc 0.8695403494176372
epoch 2 batch id 801 loss 0.2985261082649231 train acc 0.8724641073657927
epoch 2 batch id 1001 loss 0.007403121329843998 train acc 0.8894230769230769
epoch 2 batch id 1201 loss 0.3115110695362091 train acc 0.9002914238134888
epoch 2 batch id 1401 loss 0.06964480131864548 train acc 0.9008966809421841
epoch 2 train acc 0.9022643658023827


  0%|          | 0/1427 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.5090622901916504 train acc 0.875
epoch 3 batch id 201 loss 0.292111337184906 train acc 0.9095149253731343
epoch 3 batch id 401 loss 0.4444887638092041 train acc 0.902041770573566
epoch 3 batch id 601 loss 0.21023023128509521 train acc 0.906301996672213
epoch 3 batch id 801 loss 0.1527291089296341 train acc 0.9098392634207241
epoch 3 batch id 1001 loss 0.011916844174265862 train acc 0.9230457042957043
epoch 3 batch id 1201 loss 0.16866306960582733 train acc 0.9309429641965029
epoch 3 batch id 1401 loss 0.03865913301706314 train acc 0.9319013204853676
epoch 3 train acc 0.9329230903994394


  0%|          | 0/1427 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.11502917110919952 train acc 0.9375
epoch 4 batch id 201 loss 0.2961752414703369 train acc 0.9378109452736318
epoch 4 batch id 401 loss 0.2758001387119293 train acc 0.9339152119700748
epoch 4 batch id 601 loss 0.0788237527012825 train acc 0.9370320299500832
epoch 4 batch id 801 loss 0.07806512713432312 train acc 0.939411672908864
epoch 4 batch id 1001 loss 0.014260242693126202 train acc 0.948208041958042
epoch 4 batch id 1201 loss 0.06494099646806717 train acc 0.9534762697751873
epoch 4 batch id 1401 loss 0.013584853149950504 train acc 0.9535153461812991
epoch 4 train acc 0.9542966012613875


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(sentence):
  data = [sentence, '0']
  dataset_another = [data]
  logits = 0
  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
  
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)

      valid_length= valid_length
      label = label.long().to(device)

      out = model(token_ids, valid_length, segment_ids)

      for i in out:
        logits = i
        logits = logits.detach().cpu().numpy()
        logits = np.argmax(logits)

  return logits

using cached model


In [ ]:
test_df = pd.read_csv('safe/test데이터_전처리.csv')
sub_df = pd.read_csv('sample_submission.csv')

In [ ]:
pre = []

for i in tqdm(test_df['data2']):
  pre.append(predict(i))

  0%|          | 0/9131 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 9131/9131 [2:26:31<00:00,  1.04it/s]


In [ ]:
sub_df['topic_idx'] = pre
sub_df.to_csv('predict_kobert_특수문자제거.csv', index=False)